# Building MCP Servers for Compliance Documentation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ethanolivertroy/pcidss-mcp-training/blob/main/build-mcp-server.ipynb)

**A hands-on tutorial for GRC professionals**

---

## What You'll Learn

This notebook teaches you how to build a Model Context Protocol (MCP) server for compliance documentation. We use PCI-DSS as the example, but these patterns work for **any** compliance framework:

- NIST 800-53
- ISO 27001
- SOC 2
- FedRAMP
- HIPAA

### What is an MCP Server?

MCP servers provide **tools** that AI assistants like Claude can call to:
- Search structured compliance data
- Retrieve specific requirements by ID
- Map controls between frameworks
- Generate compliance checklists

### Learning Objectives

By the end of this notebook, you will:

1. **Extract Data** - Parse compliance data from Excel files
2. **Build Indexes** - Create fast O(1) lookup structures
3. **Understand MCP Patterns** - Schema validation with Zod
4. **Implement Search** - Full-text search with TF-IDF scoring
5. **Build a Complete Server** - Working TypeScript MCP server
6. **Adapt for Other Frameworks** - Apply patterns to NIST, ISO, etc.

---

## Part 1: Data Source - PCI-DSS Prioritized Approach Tool

### Why Excel Over PDF?

Many compliance documents are available as PDFs, but PDFs are difficult to parse. The same data is often available in Excel format, which is:

- **Already structured** - No complex extraction needed
- **Reliable** - Consistent column layouts
- **Fast** - Parse in milliseconds vs. minutes
- **Accurate** - No OCR errors or layout issues

For PCI-DSS, the **Prioritized Approach Tool** is an official Excel file from PCI Security Standards Council.

### Download the Excel File

1. Download from PCI SSC:
   
   **[Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx](https://docs-prv.pcisecuritystandards.org/PCI%20DSS/Supporting%20Document/Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx)**

2. Accept the license agreement

3. Upload when prompted (Colab) or place in your project directory (local)

### Legal Notes

The Excel file is licensed by PCI SSC for **internal use only**:
- Use for your organization's compliance work
- Do not redistribute converted data publicly
- Each user should download and accept the license directly

## Part 2: Parsing Excel Data

The Prioritized Approach Tool contains a sheet called "Prioritized Approach Milestones" with:

| Column A | Column B |
|----------|----------|
| Requirement text | Milestone priority (1-6) |

Requirements follow a hierarchical numbering:
- **Level 1**: "Requirement 1: Install and Maintain Network Security Controls"
- **Level 2**: "1.1 Processes and mechanisms..."
- **Level 3**: "1.1.1 All security policies..."

In [ ]:
# Install dependencies
!pip install -q openpyxl
print("Dependencies installed.")

In [ ]:
from pathlib import Path
import re
from typing import Dict, List, Any, Optional, Tuple

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running locally")

if IN_COLAB:
    from google.colab import files
    print("\nPlease upload the Prioritized Approach Tool Excel file...")
    print("Download from: https://docs-prv.pcisecuritystandards.org/PCI%20DSS/Supporting%20Document/Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx")
    uploaded = files.upload()
    excel_path = Path(list(uploaded.keys())[0])
    print(f"\nUploaded: {excel_path.name}")
else:
    excel_path = Path("Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx")
    if not excel_path.exists():
        print(f"Excel file not found. Please download from:")
        print("https://docs-prv.pcisecuritystandards.org/PCI%20DSS/Supporting%20Document/Prioritized-Approach-Tool-For-PCI-DSS-v4_0_1.xlsx")
    else:
        print(f"Found: {excel_path.name}")

In [ ]:
def parse_requirement_cell(text: str) -> Optional[Dict[str, Any]]:
    """Parse requirement ID and title from Excel cell."""
    if not text:
        return None
    
    text = str(text).strip()
    
    # Pattern: "1.1.1 Title text"
    match = re.match(r'^(\d+(?:\.\d+)*)\s+(.+)', text)
    if match:
        req_id = match.group(1)
        title = match.group(2).strip()
        level = len(req_id.split('.'))
        return {'id': req_id, 'title': title, 'level': level}
    
    # Pattern: "Requirement 1: Title"
    section_match = re.match(r'^Requirement\s+(\d+):\s*(.+)', text, re.IGNORECASE)
    if section_match:
        return {
            'id': section_match.group(1),
            'title': section_match.group(2).strip(),
            'level': 1
        }
    
    return None

# Test
test_cases = [
    "Requirement 1: Install and Maintain Network Security Controls",
    "1.1 Processes and mechanisms for network security controls",
    "1.2.3 Specific requirement text"
]
print("Testing parser:")
for text in test_cases:
    result = parse_requirement_cell(text)
    if result:
        print(f"  {result['id']}: Level {result['level']}")

In [ ]:
from openpyxl import load_workbook

def extract_requirements(excel_path: Path) -> List[Dict[str, Any]]:
    """Extract requirements from Excel file."""
    workbook = load_workbook(excel_path, data_only=True)
    
    sheet_name = "Prioritized Approach Milestones"
    if sheet_name not in workbook.sheetnames:
        print(f"Available sheets: {workbook.sheetnames}")
        return []
    
    worksheet = workbook[sheet_name]
    requirements = []
    
    for row in worksheet.iter_rows(min_row=3, values_only=True):
        if not row or not row[0]:
            continue
        
        parsed = parse_requirement_cell(str(row[0]))
        if not parsed:
            continue
        
        parts = parsed['id'].split('.')
        parent_id = '.'.join(parts[:-1]) if len(parts) > 1 else None
        
        requirement = {
            'id': parsed['id'],
            'level': parsed['level'],
            'parentId': parent_id,
            'title': parsed['title'],
            'statement': parsed['title'],
        }
        
        if row[1]:
            requirement['milestone'] = int(row[1])
        
        requirements.append(requirement)
    
    requirements.sort(key=lambda r: [int(x) for x in r['id'].split('.')])
    return requirements

# Extract
if excel_path.exists():
    requirements = extract_requirements(excel_path)
    print(f"Extracted {len(requirements)} requirements")
    
    levels = {}
    for req in requirements:
        levels[req['level']] = levels.get(req['level'], 0) + 1
    
    print("\nLevel distribution:")
    for level in sorted(levels.keys()):
        print(f"  Level {level}: {levels[level]}")
else:
    requirements = []

## Part 3: Building a Fast Index

When an AI asks "What is requirement 1.2.3?", we need O(1) lookup, not O(n) linear search.

In [ ]:
class RequirementIndex:
    """Fast O(1) index for requirement lookups."""
    
    def __init__(self):
        self.requirements: Dict[str, Dict] = {}
        self.by_level: Dict[int, List[str]] = {}
        
    def add(self, req: Dict):
        self.requirements[req['id']] = req
        level = req['level']
        if level not in self.by_level:
            self.by_level[level] = []
        self.by_level[level].append(req['id'])
    
    def get(self, req_id: str) -> Optional[Dict]:
        return self.requirements.get(req_id)
    
    def get_children(self, parent_id: str) -> List[Dict]:
        return sorted(
            [r for r in self.requirements.values() if r.get('parentId') == parent_id],
            key=lambda r: [int(x) for x in r['id'].split('.')]
        )

# Build index
if requirements:
    index = RequirementIndex()
    for req in requirements:
        index.add(req)
    print(f"Indexed {len(index.requirements)} requirements")
    
    # Test
    req = index.get("1.2")
    if req:
        print(f"\nTest lookup 1.2: {req['title'][:50]}...")
        children = index.get_children("1.2")
        print(f"Children of 1.2: {len(children)}")

## Part 4: MCP Tool Patterns

Every MCP tool follows: **Schema -> Execute -> Return**

In [ ]:
from dataclasses import dataclass

@dataclass
class GetRequirementInput:
    id: str
    include_children: bool = False

def get_requirement_tool(input: GetRequirementInput, idx: RequirementIndex):
    """MCP tool pattern: validate -> execute -> return"""
    req = idx.get(input.id)
    if not req:
        raise ValueError(f"Requirement not found: {input.id}")
    
    result = {'requirement': req}
    if input.include_children:
        result['children'] = idx.get_children(input.id)
    return result

# Test
if requirements:
    result = get_requirement_tool(GetRequirementInput(id="1", include_children=True), index)
    print(f"Requirement 1: {result['requirement']['title'][:40]}...")
    print(f"Children: {len(result['children'])}")

## Part 5: Full-Text Search

TF-IDF scoring for "find requirements about encryption" queries.

In [ ]:
from collections import Counter
import math

class SearchIndex:
    """Simple TF-IDF search."""
    
    def __init__(self, reqs: List[Dict]):
        self.requirements = {r['id']: r for r in reqs}
        self.inverted_index: Dict[str, List] = {}
        self.doc_count = len(reqs)
        
        for req in reqs:
            text = f"{req['id']} {req['title']}".lower()
            word_counts = Counter(text.split())
            for word, count in word_counts.items():
                if word not in self.inverted_index:
                    self.inverted_index[word] = []
                self.inverted_index[word].append((req['id'], count))
    
    def search(self, query: str, limit: int = 5):
        scores = {}
        for word in query.lower().split():
            if word not in self.inverted_index:
                continue
            idf = math.log(self.doc_count / len(self.inverted_index[word]))
            for doc_id, tf in self.inverted_index[word]:
                scores[doc_id] = scores.get(doc_id, 0) + tf * idf
        
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)[:limit]

# Test
if requirements:
    search_index = SearchIndex(requirements)
    results = search_index.search("network security firewall")
    print("Search: 'network security firewall'")
    for req_id, score in results:
        print(f"  [{req_id}] {search_index.requirements[req_id]['title'][:50]}...")

## Part 6: Save Data as JSON

Export for use in the MCP server.

In [ ]:
import json
from datetime import datetime

if requirements:
    output = {
        'info': {
            'version': '4.0.1',
            'convertedAt': datetime.now().isoformat(),
        },
        'requirements': requirements,
    }
    
    output_file = Path('requirements.json')
    with open(output_file, 'w') as f:
        json.dump(output, f, indent=2)
    
    print(f"Saved: {output_file} ({output_file.stat().st_size / 1024:.1f} KB)")
    print(f"Requirements: {len(requirements)}")
    
    if IN_COLAB:
        files.download(str(output_file))

## Part 7: Adapting for Other Frameworks

The same patterns work for any compliance framework:

### NIST 800-53
```python
# Pattern: AC-1, AC-1(1)
match = re.match(r'^([A-Z]{2}-\d+(?:\(\d+\))?)\s+(.+)$', text)
```

### ISO 27001
```python
# Pattern: A.5.1.1
match = re.match(r'^(A\.\d+(?:\.\d+)*)\s+(.+)$', text)
```

### Key Steps
1. Find structured data (Excel/CSV > PDF)
2. Write regex for your ID format
3. Build the same index structure
4. Adapt tool descriptions

## Part 8: Building the Complete MCP Server

Now let's build a working MCP server in TypeScript. You'll need:
- Node.js 18+
- npm

### Project Setup

Create a new directory and initialize:

```bash
mkdir my-compliance-mcp
cd my-compliance-mcp
npm init -y
```

### package.json

```json
{
  "name": "compliance-mcp-server",
  "version": "1.0.0",
  "type": "module",
  "main": "dist/index.js",
  "scripts": {
    "build": "tsc",
    "dev": "node dist/index.js"
  },
  "dependencies": {
    "@modelcontextprotocol/sdk": "^1.0.0",
    "zod": "^3.22.0"
  },
  "devDependencies": {
    "typescript": "^5.0.0",
    "@types/node": "^20.0.0"
  }
}
```

Install dependencies:
```bash
npm install
```

### tsconfig.json

```json
{
  "compilerOptions": {
    "target": "ES2022",
    "module": "NodeNext",
    "moduleResolution": "NodeNext",
    "outDir": "./dist",
    "rootDir": "./src",
    "strict": true,
    "esModuleInterop": true,
    "skipLibCheck": true
  },
  "include": ["src/**/*"]
}
```

### src/index.ts - Complete MCP Server

```typescript
#!/usr/bin/env node
import { McpServer } from "@modelcontextprotocol/sdk/server/mcp.js";
import { StdioServerTransport } from "@modelcontextprotocol/sdk/server/stdio.js";
import { z } from "zod";
import { readFileSync } from "fs";

// =============================================================================
// Types
// =============================================================================

interface Requirement {
  id: string;
  level: number;
  parentId: string | null;
  title: string;
  statement: string;
  milestone?: number;
}

interface RequirementData {
  info: { version: string };
  requirements: Requirement[];
}

// =============================================================================
// Index - O(1) Lookups
// =============================================================================

class RequirementIndex {
  private byId = new Map<string, Requirement>();
  private byLevel = new Map<number, string[]>();

  constructor(requirements: Requirement[]) {
    for (const req of requirements) {
      this.byId.set(req.id, req);
      const levelList = this.byLevel.get(req.level) || [];
      levelList.push(req.id);
      this.byLevel.set(req.level, levelList);
    }
  }

  get(id: string): Requirement | undefined {
    return this.byId.get(id);
  }

  getByLevel(level: number): Requirement[] {
    const ids = this.byLevel.get(level) || [];
    return ids.map(id => this.byId.get(id)!).filter(Boolean);
  }

  getChildren(parentId: string): Requirement[] {
    return Array.from(this.byId.values())
      .filter(r => r.parentId === parentId)
      .sort((a, b) => a.id.localeCompare(b.id, undefined, { numeric: true }));
  }

  search(query: string, limit = 10): Requirement[] {
    const terms = query.toLowerCase().split(/\s+/);
    const scores = new Map<string, number>();

    for (const [id, req] of this.byId) {
      const text = `${req.id} ${req.title} ${req.statement}`.toLowerCase();
      let score = 0;
      for (const term of terms) {
        if (text.includes(term)) score++;
      }
      if (score > 0) scores.set(id, score);
    }

    return Array.from(scores.entries())
      .sort((a, b) => b[1] - a[1])
      .slice(0, limit)
      .map(([id]) => this.byId.get(id)!);
  }

  all(): Requirement[] {
    return Array.from(this.byId.values());
  }
}

// =============================================================================
// Load Data
// =============================================================================

function loadRequirements(path: string): RequirementIndex {
  const data: RequirementData = JSON.parse(readFileSync(path, "utf-8"));
  return new RequirementIndex(data.requirements);
}

// =============================================================================
// MCP Server
// =============================================================================

async function main() {
  // Load requirements from JSON (generated by Part 6)
  const index = loadRequirements("./requirements.json");
  console.error(`Loaded ${index.all().length} requirements`);

  // Create MCP server
  const server = new McpServer({
    name: "compliance-server",
    version: "1.0.0",
  });

  // -------------------------------------------------------------------------
  // Tool: get_requirement
  // -------------------------------------------------------------------------
  server.tool(
    "get_requirement",
    "Get a specific requirement by ID",
    {
      id: z.string().describe("Requirement ID (e.g., '1.2.3')"),
      includeChildren: z.boolean().default(false).describe("Include child requirements"),
    },
    async ({ id, includeChildren }) => {
      const requirement = index.get(id);
      if (!requirement) {
        return {
          content: [{ type: "text", text: JSON.stringify({ error: `Not found: ${id}` }) }],
          isError: true,
        };
      }

      const result: any = { requirement };
      if (includeChildren) {
        result.children = index.getChildren(id);
      }

      return {
        content: [{ type: "text", text: JSON.stringify(result, null, 2) }],
      };
    }
  );

  // -------------------------------------------------------------------------
  // Tool: search_requirements
  // -------------------------------------------------------------------------
  server.tool(
    "search_requirements",
    "Search requirements by keyword",
    {
      query: z.string().describe("Search query"),
      limit: z.number().default(10).describe("Max results"),
    },
    async ({ query, limit }) => {
      const results = index.search(query, limit);
      return {
        content: [{
          type: "text",
          text: JSON.stringify({ query, count: results.length, results }, null, 2),
        }],
      };
    }
  );

  // -------------------------------------------------------------------------
  // Tool: list_requirements
  // -------------------------------------------------------------------------
  server.tool(
    "list_requirements",
    "List requirements, optionally filtered by level",
    {
      level: z.number().optional().describe("Filter by level (1, 2, 3, etc.)"),
      limit: z.number().default(20).describe("Max results"),
    },
    async ({ level, limit }) => {
      let results = level ? index.getByLevel(level) : index.all();
      results = results.slice(0, limit);

      return {
        content: [{
          type: "text",
          text: JSON.stringify({
            level: level || "all",
            count: results.length,
            requirements: results.map(r => ({ id: r.id, title: r.title })),
          }, null, 2),
        }],
      };
    }
  );

  // Connect via stdio
  const transport = new StdioServerTransport();
  await server.connect(transport);
  console.error("MCP server running");
}

main().catch(console.error);
```

### Build and Run

```bash
# Build TypeScript
npm run build

# Run server (for testing)
npm run dev
```

### Configure Claude Desktop

Add to `~/.claude/claude_desktop_config.json`:

```json
{
  "mcpServers": {
    "compliance": {
      "command": "node",
      "args": ["/path/to/my-compliance-mcp/dist/index.js"]
    }
  }
}
```

Restart Claude Desktop and your tools will be available!

## Summary

You've learned to build MCP servers for compliance documentation:

1. **Data Extraction** - Excel parsing with openpyxl
2. **Indexing** - O(1) Map lookups
3. **MCP Tool Patterns** - Schema -> Execute -> Return
4. **Full-Text Search** - TF-IDF scoring
5. **Complete Server** - Working TypeScript implementation
6. **Adaptation** - Same patterns for any framework

### Resources

- [MCP Specification](https://modelcontextprotocol.io/specification/2025-11-25)
- [MCP TypeScript SDK](https://github.com/modelcontextprotocol/typescript-sdk)
- [This Tutorial](https://github.com/ethanolivertroy/pcidss-mcp-training)

---

**Go build something!**